In [1]:
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,random_split
from torchvision import datasets, transforms

import pandas as pd
import numpy as np

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [3]:
data = pd.read_csv("dummyDataTopBottom0.csv",header = None)
x=data.iloc[1:,1:5].astype('float32')
y=data.iloc[1:,-1].astype('float32')
x.iloc[:, 2] = x.iloc[:, 2] / 100
x.iloc[:, 3] = x.iloc[:, 3] / 100
y = y/max(y.values)

In [4]:
y

1         0.566667
2         0.700000
3         0.000000
4         0.000000
5         0.000000
            ...   
99996     0.000000
99997     0.566667
99998     0.000000
99999     0.000000
100000    0.800000
Name: 5, Length: 100000, dtype: float32

In [5]:
class WaterLevelDataset(Dataset):
    def __init__(self, x, y):
        x= torch.tensor(x,dtype=torch.float32)
        y = torch.tensor(y,dtype=torch.float32)
        self.X = x
        self.Y = y

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.Y[idx]]

In [6]:
dataset = WaterLevelDataset(x.values,y.values)

In [7]:
len(dataset.X)

100000

In [8]:
dataset.X

tensor([[1.0000, 0.0000, 0.9000, 0.3500],
        [0.0000, 1.0000, 0.4300, 0.1200],
        [0.0000, 1.0000, 0.1200, 0.2200],
        ...,
        [0.0000, 0.0000, 0.1600, 0.9000],
        [1.0000, 0.0000, 0.2100, 0.1600],
        [0.0000, 0.0000, 0.3800, 0.3900]])

In [9]:
dataset.Y

tensor([0.5667, 0.7000, 0.0000,  ..., 0.0000, 0.0000, 0.8000])

In [10]:
train, test = random_split(dataset, [int(len(dataset.X)*0.8),int(len(dataset.X)*0.2)])

In [11]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

## Model 1
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [12]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=self.layer2(X)
        X = self.activation(X)
        return X

# Instantiate the model
model = SimpleNN()



In [13]:
loss_fn = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=0.001)

In [14]:
for epoch in range(250):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        outputs = model(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/250], Loss: {loss.item()*100:0f}")



Epoch [1/100], Loss: 11.851054
Epoch [2/100], Loss: 16.518635
Epoch [3/100], Loss: 6.931745
Epoch [4/100], Loss: 14.177941
Epoch [5/100], Loss: 13.740896
Epoch [6/100], Loss: 9.648876
Epoch [7/100], Loss: 10.867815
Epoch [8/100], Loss: 7.120141
Epoch [9/100], Loss: 10.616314
Epoch [10/100], Loss: 8.228975
Epoch [11/100], Loss: 7.369996
Epoch [12/100], Loss: 8.266341
Epoch [13/100], Loss: 11.442620
Epoch [14/100], Loss: 8.651751
Epoch [15/100], Loss: 6.754599
Epoch [16/100], Loss: 11.642706
Epoch [17/100], Loss: 10.764017
Epoch [18/100], Loss: 7.982962
Epoch [19/100], Loss: 9.153353
Epoch [20/100], Loss: 8.693673
Epoch [21/100], Loss: 13.126782
Epoch [22/100], Loss: 9.150416
Epoch [23/100], Loss: 9.438281
Epoch [24/100], Loss: 10.527661
Epoch [25/100], Loss: 9.684453
Epoch [26/100], Loss: 12.720722
Epoch [27/100], Loss: 5.440531
Epoch [28/100], Loss: 7.418154
Epoch [29/100], Loss: 8.677002
Epoch [30/100], Loss: 7.391369
Epoch [31/100], Loss: 6.686731
Epoch [32/100], Loss: 5.752387
Epoch

In [15]:
model.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            outputs = model(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx])
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {outputs.item()*30:.2f}, Actual: {targets[idx].item()*30:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")


Predicted: 14.51, Actual: 0.00
Predicted: 12.03, Actual: 4.00
Predicted: 16.02, Actual: 26.00
Predicted: 15.26, Actual: 19.00
Predicted: 4.79, Actual: 0.00
Predicted: 15.81, Actual: 28.00
Predicted: 5.01, Actual: 0.00
Predicted: 6.00, Actual: 0.00
Predicted: 3.00, Actual: 0.00
Predicted: 0.79, Actual: 0.00
Predicted: 8.65, Actual: 0.00
Predicted: 3.46, Actual: 0.00
Predicted: 3.45, Actual: 0.00
Predicted: 5.97, Actual: 0.00
Predicted: 16.90, Actual: 0.00
Predicted: 4.61, Actual: 0.00
Average loss on the testing dataset: 7.4439
Predicted: 1.65, Actual: 0.00
Predicted: 5.54, Actual: 0.00
Predicted: 9.48, Actual: 0.00
Predicted: 13.48, Actual: 13.00
Predicted: 1.88, Actual: 0.00
Predicted: 15.38, Actual: 25.00
Predicted: 12.81, Actual: 8.00
Predicted: 1.69, Actual: 0.00
Predicted: 0.17, Actual: 0.00
Predicted: 2.18, Actual: 0.00
Predicted: 4.24, Actual: 0.00
Predicted: 10.91, Actual: 18.00
Predicted: 0.53, Actual: 0.00
Predicted: 0.59, Actual: 0.00
Predicted: 1.69, Actual: 0.00
Predicted:

In [16]:
torch.save(model, "model/model1TopBotomData0-25.h5")

## Model 2
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [17]:
class Model2NN(nn.Module):
    def __init__(self):
        super(Model2NN, self).__init__()
        self.layer = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10, 10)
        self.layer3 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=torch.relu(self.layer2(X))
        X=self.layer3(X)
        X = self.activation(X)
        return X

# Instantiate the model
model1 = Model2NN()



In [18]:
loss_fn = nn.MSELoss()

optimizer = optim.SGD(model1.parameters(), lr=0.001)

In [19]:
for epoch in range(250):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        outputs = model1(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/250], Loss: {loss.item()*100:0f}")



Epoch [1/100], Loss: 7.986596
Epoch [2/100], Loss: 10.176009
Epoch [3/100], Loss: 8.049957
Epoch [4/100], Loss: 9.934926
Epoch [5/100], Loss: 6.122306
Epoch [6/100], Loss: 12.215233
Epoch [7/100], Loss: 16.500826
Epoch [8/100], Loss: 9.227470
Epoch [9/100], Loss: 8.701928
Epoch [10/100], Loss: 14.663412
Epoch [11/100], Loss: 9.060102
Epoch [12/100], Loss: 11.665813
Epoch [13/100], Loss: 7.505143
Epoch [14/100], Loss: 10.457563
Epoch [15/100], Loss: 6.584811
Epoch [16/100], Loss: 7.953745
Epoch [17/100], Loss: 7.225863
Epoch [18/100], Loss: 6.437480
Epoch [19/100], Loss: 13.060673
Epoch [20/100], Loss: 15.143611
Epoch [21/100], Loss: 5.612498
Epoch [22/100], Loss: 11.435847
Epoch [23/100], Loss: 5.891475
Epoch [24/100], Loss: 6.364007
Epoch [25/100], Loss: 6.157279
Epoch [26/100], Loss: 6.316663
Epoch [27/100], Loss: 12.673733
Epoch [28/100], Loss: 8.160020
Epoch [29/100], Loss: 11.371556
Epoch [30/100], Loss: 8.537097
Epoch [31/100], Loss: 10.955738
Epoch [32/100], Loss: 5.290784
Epoch

In [40]:
model1.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            print(test_inputs)
            outputs = model1(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx])
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {outputs.item()*30:.2f}, Actual: {targets[idx].item()*30:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")

tensor([0.0000, 1.0000, 0.8100, 0.4100])
Predicted: 15.25, Actual: 0.00
tensor([1.0000, 1.0000, 0.4400, 0.1400])
Predicted: 14.16, Actual: 4.00
tensor([0.0000, 0.0000, 0.8900, 0.0100])
Predicted: 15.21, Actual: 26.00
tensor([1.0000, 0.0000, 0.7700, 0.3200])
Predicted: 15.24, Actual: 19.00
tensor([0.0000, 1.0000, 0.2000, 0.2800])
Predicted: 0.85, Actual: 0.00
tensor([1.0000, 1.0000, 0.5800, 0.1200])
Predicted: 14.81, Actual: 28.00
tensor([1.0000, 0.0000, 0.1900, 0.2300])
Predicted: 0.77, Actual: 0.00
tensor([1.0000, 1.0000, 0.2300, 0.1300])
Predicted: 1.63, Actual: 0.00
tensor([1.0000, 1.0000, 0.9100, 0.9600])
Predicted: 0.05, Actual: 0.00
tensor([0.0000, 0.0000, 0.3000, 0.9400])
Predicted: 0.04, Actual: 0.00
tensor([1.0000, 0.0000, 0.9500, 0.7400])
Predicted: 3.46, Actual: 0.00
tensor([0.0000, 1.0000, 0.1300, 0.2200])
Predicted: 0.18, Actual: 0.00
tensor([1.0000, 1.0000, 0.3900, 0.7500])
Predicted: 2.86, Actual: 0.00
tensor([1.0000, 1.0000, 0.8600, 0.8000])
Predicted: 1.16, Actual: 0.0

In [21]:
torch.save(model1, "model/model2TopBotomData0-25.h5")

In [57]:
test1 = torch.tensor([1,0,90/100,20/100])
print(test1)
outputs = model1(test1)
print(int(outputs.item()*30))

tensor([1.0000, 0.0000, 0.9000, 0.2000])
15
